In [156]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import requests

In [157]:
#Path to chromedriver
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [129]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

In [130]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [131]:
# Extract latest news article
article = soup.find('div', class_="list_text")

In [105]:
article

<div class="list_text"><div class="list_date">August 24, 2020</div><div class="content_title"><a href="/news/8744/nasa-engineers-checking-insights-weather-sensors/" target="_self">NASA Engineers Checking InSight's Weather Sensors</a></div><div class="article_teaser_body">An electronics issue is suspected to be preventing the sensors from sharing their data about Mars weather with the spacecraft.</div></div>

In [132]:
#Create variables "news_title" and "news_p" to save the info
news_title = article.find('div', class_="content_title").text  
news_p = article.find('div', class_="article_teaser_body").text                

In [133]:
news_title

"NASA Engineers Checking InSight's Weather Sensors"

In [134]:
news_p

'An electronics issue is suspected to be preventing the sensors from sharing their data about Mars weather with the spacecraft.'

In [136]:
#Visit the url for JPL Featured Space Image
url2 = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url2)

In [137]:
#Use splinter to navigate the site and find the image url 
#for the current Featured Mars Image and assign the url string to a variable called featured_image_url
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
image= soup.find('img', class_="thumb")["src"]

In [138]:
image

'/spaceimages/images/wallpaper/PIA24077-640x350.jpg'

In [163]:
#Get the complete url to the jpg
featured_image_url="https://jpl.nasa.gov"+ image

In [140]:
#Display the complete url
featured_image_url

'https://jpl.nasa.gov/spaceimages/images/wallpaper/PIA24077-640x350.jpg'

In [141]:
#Go to the website to scrape information about mars
url3 = "https://space-facts.com/mars/"
browser.visit(url3)

In [142]:
#Use Pandas to convert the data to a HTML table string
#Start by saving the data into a dataframe and then output it to a HTML table
import pandas as pd

In [143]:
#Grab the data from the website and save it as a dataframe
grab=pd.read_html(url3)
mars_data=pd.DataFrame(grab[0])
mars_data.columns=['Mars','Data']
mars_table=mars_data.set_index("Mars")

In [144]:
#View the data
mars_data

,Mars,Data
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [167]:
#Convert the dataframe to html 
mars_html = mars_table.to_html(classes='marsdata')
#Output
print(mars_html)
mars_html.to_html("mars_table.html")

<table border="1" class="dataframe marsdata">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Data</th>
    </tr>
    <tr>
      <th>Mars</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


AttributeError: 'str' object has no attribute 'to_html'

In [161]:
#Visit the USGS Astrogeology site to obtain high resolution images for each of Mar's hemispheres.
url5 = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url5)

In [162]:
# HTML Object
html_hemispheres = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html_hemispheres, 'html.parser')

# Retreive all items that contain mars hemispheres information
items = soup.find_all('div', class_='item')

# Create empty list for hemisphere urls 
hemisphere_image_urls = []

# Store the main_ul 
hemispheres_main_url = 'https://astrogeology.usgs.gov'

# Loop through the items previously stored
for i in items: 
    # Store title
    title = i.find('h3').text
    
    # Store link that leads to full image website
    partial_img_url = i.find('a', class_='itemLink product-item')['href']
    
    # Visit the link that contains the full image website 
    browser.visit(hemispheres_main_url + partial_img_url)
    
    # HTML Object of individual hemisphere information website 
    partial_img_html = browser.html
    
    # Parse HTML with Beautiful Soup for every individual hemisphere information website 
    soup = BeautifulSoup( partial_img_html, 'html.parser')
    
    # Retrieve full image source 
    img_url = hemispheres_main_url + soup.find('img', class_='wide-image')['src']
    
    # Append the retreived information into a list of dictionaries 
    hemisphere_image_urls.append({"title" : title, "img_url" : img_url})
    

# Display hemisphere_image_urls
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]